In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r'C:\Users\sachi\OneDrive\Product Demand Forecasting\DATA\sales_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ProductID      300000 non-null  object
 1   Location       300000 non-null  object
 2   Date           300000 non-null  object
 3   UnitsSold      300000 non-null  int64 
 4   PromoCodeUsed  295000 non-null  object
dtypes: int64(1), object(4)
memory usage: 11.4+ MB


In [4]:
df.sample(10)

,ProductID,Location,Date,UnitsSold,PromoCodeUsed
219582,P0366,L10,2024-02-12,4,No
144314,P0241,L06,2024-01-15,6,Yes
255840,P0427,L05,2024-01-01,4,No
84694,P0142,L02,2024-02-04,6,Yes
228438,P0381,L08,2024-01-19,2,No
273830,P0457,L04,2024-02-20,1,Yes
141413,P0236,L07,2024-02-23,3,No
62988,P0105,L10,2024-02-18,6,Yes
128552,P0215,L03,2024-02-02,2,Yes
192354,P0321,L06,2024-02-24,0,No


In [5]:
df.nunique()

ProductID        500
Location          10
Date              60
UnitsSold         18
PromoCodeUsed      2
dtype: int64

In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
df.isnull().sum()

ProductID           0
Location            0
Date                0
UnitsSold           0
PromoCodeUsed    5000
dtype: int64

# filling missing data of PromoCodeUsed column

In [9]:
# logic is that, extracting 100 data of PromoCodeUSed for each productId , counting the number of "no" and "yes".. fill the value with most number
sampled_df = df.groupby('ProductID').apply(lambda x: x.sample(min(len(x), 100))).reset_index(drop=True)


C:\Users\sachi\AppData\Local\Temp\ipykernel_17272\3544640990.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('ProductID').apply(lambda x: x.sample(min(len(x), 100))).reset_index(drop=True)


In [11]:
promo_counts = sampled_df.groupby(['ProductID', 'PromoCodeUsed']).size().unstack(fill_value=0)


In [12]:
# Create a dictionary with ProductID as key and majority promoCodeUsed as value
promo_majority = {}

for pid in promo_counts.index:
    if promo_counts.loc[pid, 'Yes'] > promo_counts.loc[pid, 'No']:
        promo_majority[pid] = 'Yes'
    else:
        promo_majority[pid] = 'No'


In [14]:
def fill_promo(row):
    if pd.isnull(row['PromoCodeUsed']):
        return promo_majority.get(row['ProductID'], 'No')  # Default 'No' if ProductID not found
    else:
        return row['PromoCodeUsed']

df['PromoCodeUsed'] = df.apply(fill_promo, axis=1)


In [16]:
print("Missing promoCodeUsed after fill:", df['PromoCodeUsed'].isnull().sum())


Missing promoCodeUsed after fill: 0


In [17]:
df.isnull().sum()

ProductID        0
Location         0
Date             0
UnitsSold        0
PromoCodeUsed    0
dtype: int64

In [19]:
df.duplicated().sum()

np.int64(0)

In [20]:
df.sample()

,ProductID,Location,Date,UnitsSold,PromoCodeUsed
289892,P0484,L02,2024-02-02,4,No


In [22]:
df.dtypes

ProductID        object
Location         object
Date             object
UnitsSold         int64
PromoCodeUsed    object
dtype: object

In [23]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [24]:
df.dtypes

ProductID                object
Location                 object
Date             datetime64[ns]
UnitsSold                 int64
PromoCodeUsed            object
dtype: object

In [32]:
negative_units = df[df['UnitsSold'] < 0]

print(f"Total negative UnitsSold rows: {len(negative_units)}")

Total negative UnitsSold rows: 0


# outliers

In [33]:
Q1 = df['UnitsSold'].quantile(0.25)
Q3 = df['UnitsSold'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Outlier rows
outliers = df[(df['UnitsSold'] < lower_bound) | (df['UnitsSold'] > upper_bound)]

print(f"Total outliers in UnitsSold: {len(outliers)}")


Total outliers in UnitsSold: 5667


In [34]:
outliers.sort_values(by='UnitsSold', ascending=False).head(10)


,ProductID,Location,Date,UnitsSold,PromoCodeUsed
189975,P0317,L07,2024-01-16,17,Yes
224605,P0375,L04,2024-01-26,16,No
93820,P0157,L04,2024-02-10,16,No
195097,P0326,L02,2024-02-07,16,Yes
90260,P0151,L05,2024-01-21,15,No
100210,P0168,L01,2024-01-11,15,No
62346,P0104,L10,2024-01-07,15,Yes
60488,P0101,L09,2024-01-09,15,No
263035,P0439,L04,2024-02-25,15,No
267404,P0446,L07,2024-02-14,15,Yes


In [35]:
outliers['ProductID'].value_counts().head(10)


ProductID
P0065    21
P0159    21
P0072    20
P0172    20
P0450    20
P0206    20
P0221    20
P0026    19
P0489    19
P0240    19
Name: count, dtype: int64

In [36]:
df.describe()

,Date,UnitsSold
count,300000,300000.000000
mean,2024-01-30 12:00:00,3.000777
min,2024-01-01 00:00:00,0.000000
25%,2024-01-15 18:00:00,1.000000
50%,2024-01-30 12:00:00,3.000000
75%,2024-02-14 06:00:00,4.000000
max,2024-02-29 00:00:00,17.000000
std,NaN,2.238309


In [37]:
df.to_csv(r"C:\Users\sachi\OneDrive\Product Demand Forecasting\CLEANED DATA\sales_data_Cleaned.csv",index=False)